In [ ]:
def _to_keys(key: str, values: Dict) -> Dict:
    all_vals = {
        f'{key}_int': values['intercept'],
        f'{key}_lbs_z': values['lbs_z'],
        f'{key}_att_log_n': values['logNthFightAtt'],
        f'{key}_att_age_z': values['fighterAge_z'],
        f'{key}_att_age_sq_z': values['fighterAgeSq_z'],
    }
    if key not in BINOMIAL_NODST_MODEL_TO_YPRED_N:
        all_vals.update({
            f'{key}_def_log_n': values['logNthFightDef'],
            f'{key}_def_age_z': values['opponentAge_z'],
            f'{key}_def_age_sq_z': values['opponentAgeSq_z'],
        })
    return {k: v for k, v in all_vals.items()}


param_to_value = {
    k: v
    for key, values in params.items()
    for k, v in _to_keys(key, values).items()
}

to_print = [
    'ModelWeights {',
    *[f'    {k}: {float(v)},' for k, v in param_to_value.items()],
    '}'
]
print('\n'.join(to_print))

In [ ]:
dc = Counter()
new_params = load_params()
for c in loc_cols:
    cm = c.split('_')[0]
    dc[cm] += means[c]
for cm in dc:
    new_params[cm]['intercept'] += dc[cm]
# print(json.dumps(dc, indent=4))
# print(json.dumps({m: {k: round(v,6) for k,v in p.items()} for m, p in new_params.items()}, indent=4))

In [ ]:
df = load_full_dataset()
df = df[df['date'] > pandas.Timestamp('2011-11-19')]

variables = sorted(
    {
        v
        for vs in {
            **BINOMIAL_MODELS,
            **POISSON_MODEL_TO_YPRED_TIME,
        }.values()
        for v in vs
    }
)
prior_values = df[variables].mean().to_dict()
model_values = {}
for model, (ypred, n) in BINOMIAL_MODELS.items():
    model_values[model] = prior_values[ypred] / prior_values[n]
for model, (ypred, xtime) in POISSON_MODEL_TO_YPRED_TIME.items():
    model_values[model] = 60 * prior_values[ypred] / prior_values[xtime]
    
prior_pm = {p: 60 * v / prior_values['T'] for p, v in prior_values.items()}
prior_pm

prior_values['T'] / 60
df.head()
df[df['fighterId'] == 'f14cf73e51b29254'][['fightId', 'date', 'HL','SA','GBL', 'GBA', 'SBL','SBA','SHL','SHA']]